In [1]:
import os
import shutil
import numpy as np

import tensorflow as tf
from tensorflow import keras

from pathlib import Path
from IPython.display import display, Audio

# Traigo los audios desde https://www.kaggle.com/kongaevans/speaker-recognition-dataset/download
# y los guardo en la carpeta Descargas de mi directorio HOME
DATASET_ROOT = os.path.join(os.path.expanduser("~"), "Downloads/16000_pcm_speeches")

# Las carpetas en las cuales voy a poner los ejemplos de audio y los ejemplos de ruidos
AUDIO_SUBFOLDER = "audio"
NOISE_SUBFOLDER = "noise"

DATASET_AUDIO_PATH = os.path.join(DATASET_ROOT, AUDIO_SUBFOLDER)
DATASET_NOISE_PATH = os.path.join(DATASET_ROOT, NOISE_SUBFOLDER)

# Porcentaje de muestras que voy a usar para validación
VALID_SPLIT = 0.1

# Semilla que voy a usar para mezclar los datos con el ruido
SHUFFLE_SEED = 43

# La tasa de muestreo a usar es única para todas las muestras de audio.
# Se vuelve a muestrear todo el ruido a esta frecuencia de muestreo.
# Este también será el tamaño de salida de las muestras de las señales de audio.
# (ya que todas las muestras son de 1 segundo de duración)
SAMPLING_RATE = 16000

# El factor a multiplicar por el ruido, es acorde a:
#   muestra_de_ruido = muestreo + ruido * prop * escala
#      donde prop = amplitud_de_muestreo / amplitud_de_ruido
SCALE = 0.5

BATCH_SIZE = 128
EPOCHS = 100


In [2]:
# Si la carpeta 'audio' no existe, la creo; caso contrario, no hago nada.
if os.path.exists(DATASET_AUDIO_PATH) is False:
    os.makedirs(DATASET_AUDIO_PATH)

# si la carpeta 'noise' no existe, la creo; caso contrario, no hago nada.
if os.path.exists(DATASET_NOISE_PATH) is False:
    os.makedirs(DATASET_NOISE_PATH)

for folder in os.listdir(DATASET_ROOT):
    if os.path.isdir(os.path.join(DATASET_ROOT, folder)):
        if folder in [AUDIO_SUBFOLDER, NOISE_SUBFOLDER]:
            # Si la carpeta es 'audio' o 'noise', no hago nada
            continue
        elif folder in ["other", "_background_noise_"]:
            # Si la sub-carpeta es una de las que contienen muestras de ruido,
            # moverla a la carpeta 'noise'
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_NOISE_PATH, folder),
            )
        else:
            # De otra forma, debe ser una carpeta de un hablante, asique hay que moverla a la carpeta de 'audio'
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_AUDIO_PATH, folder),
            )

In [3]:
# Obtengo la lista de todos los archivos de ruido
noise_paths = []
for subdir in os.listdir(DATASET_NOISE_PATH):
    subdir_path = Path(DATASET_NOISE_PATH) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

print(
    "Found {} files belonging to {} directories".format(
        len(noise_paths), len(os.listdir(DATASET_NOISE_PATH))
    )
)

Found 6 files belonging to 2 directories


In [4]:
command = (
    "for dir in `ls -1 " + DATASET_NOISE_PATH + "`; do "
    "for file in `ls -1 " + DATASET_NOISE_PATH + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)
os.system(command)

# Dividir el ruido en fragmentos de 16000 pasos cada uno
def load_noise_sample(path):
    
    print(path)
    
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate != SAMPLING_RATE:
        # Número de cortes de 16000 cada uno, que se pueden generar a partir de la muestra de ruido
        slices = int(sample.shape[0] / SAMPLING_RATE)
        sample = tf.split(sample[: slices * SAMPLING_RATE], slices)
        return sample
    else:
        print("La tasa de muestrea para {} es incorrecta. Se ignora".format(path))
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

print(
    "{} archivos de ruido fueron divididos en {} muestras de ruido, donde cada una tiene {} seg. de tiempo.".format(
        len(noise_paths),noises.shape[0],noises.shape[1] // SAMPLING_RATE)
)

C:\Users\migue\Downloads\16000_pcm_speeches\noise\other\exercise_bike.wav
C:\Users\migue\Downloads\16000_pcm_speeches\noise\other\pink_noise.wav
C:\Users\migue\Downloads\16000_pcm_speeches\noise\_background_noise_\10convert.com_Audience-Claps_daSG5fwdA7o.wav
C:\Users\migue\Downloads\16000_pcm_speeches\noise\_background_noise_\doing_the_dishes.wav
C:\Users\migue\Downloads\16000_pcm_speeches\noise\_background_noise_\dude_miaowing.wav
C:\Users\migue\Downloads\16000_pcm_speeches\noise\_background_noise_\running_tap.wav
6 noise files were split into 510 noise samples where each is 1 sec. long


In [5]:

def paths_and_labels_to_dataset(audio_paths, labels):
    """Construir un dataset de audios y etiquetas."""
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))


def path_to_audio(path):
    """Leer y decodificar un archivo de audio."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio


def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        # Crear un tensor aleatorio del mismo tamaño que el audio que va 
        # desde 0 hasta la cantidad de muestras de flujo de ruido que tenemos.
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        # Obtener la proporción de amplitud entre el audio y el ruido.
        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        # Agregar el ruido reescalado al audio
        audio = audio + noise * prop * scale

    return audio


def audio_to_fft(audio):
    # Dado que tf.signal.fft aplica FFT en la dimensión más interna, 
    # debemos comprimir las dimensiones y luego expandirlas nuevamente 
    # después de FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Devuelve el valor absoluto de la primera mitad de la FFT 
    # que representa las frecuencias positivas
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])


# Obtener la lista de rutas de archivos de audio junto con sus etiquetas correspondientes

class_names = os.listdir(DATASET_AUDIO_PATH)
print("Nombres de nuestras Clases: {}".format(class_names,))

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Procesando al hablante {}".format(name,))
    dir_path = Path(DATASET_AUDIO_PATH) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

print(
    "Encontrados {} archivos, pertenecientes a {} clases.".format(len(audio_paths), len(class_names))
)

# Mezclado
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

# Dividir entre entrenamiento y validación
num_val_samples = int(VALID_SPLIT * len(audio_paths))
print("Usando {} archivos para entrenamiento.".format(len(audio_paths) - num_val_samples))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]

print("Usando {} archivos para validación.".format(num_val_samples))
valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

# Crear 2 datasets, uno para entrenamiento y otro para validación
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)


# Agrego el ruido al conjunto de entrenamiento
train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=SCALE), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Transformo las señales de audio a la frecuencia de dominio, usando 'audio_to_fft'
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

Our class names: ['Benjamin_Netanyau', 'Jens_Stoltenberg', 'Julia_Gillard', 'Magaret_Tarcher', 'Nelson_Mandela']
Processing speaker Benjamin_Netanyau
Processing speaker Jens_Stoltenberg
Processing speaker Julia_Gillard
Processing speaker Magaret_Tarcher
Processing speaker Nelson_Mandela
Found 7501 files belonging to 5 classes.
Using 6751 files for training.
Using 750 files for validation.


In [6]:

def residual_block(x, filters, conv_num=3, activation="relu"):
    # Atajo
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((SAMPLING_RATE // 2, 1), len(class_names))

model.summary()

# Compilar el modelo, usando la tasa de aprendizaje por defecto de Adam.
model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Agregar devoluciones de llamada:
# 'EarlyStopping' para dejar de entrenar cuando el modelo ya no mejora.
# 'ModelCheckPoint' mantener siempre el modelo que tiene el mejor val_accuracy
model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000, 1)]    0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 8000, 16)     64          ['input[0][0]']                  
                                                                                                  
 activation (Activation)        (None, 8000, 16)     0           ['conv1d_1[0][0]']               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 8000, 16)     784         ['activation[0][0]']             
                                                                                              

In [7]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/100
53/53 [==============================] - 876s 16s/step - loss: 1.4908 - accuracy: 0.6731 - val_loss: 0.3472 - val_accuracy: 0.8627
Epoch 2/100
53/53 [==============================] - 848s 16s/step - loss: 0.2592 - accuracy: 0.9037 - val_loss: 0.2871 - val_accuracy: 0.8880
Epoch 3/100
53/53 [==============================] - 892s 17s/step - loss: 0.1894 - accuracy: 0.9253 - val_loss: 0.1457 - val_accuracy: 0.9320
Epoch 4/100
53/53 [==============================] - 925s 17s/step - loss: 0.1544 - accuracy: 0.9387 - val_loss: 0.1113 - val_accuracy: 0.9573
Epoch 5/100
53/53 [==============================] - 781s 15s/step - loss: 0.1510 - accuracy: 0.9418 - val_loss: 0.1590 - val_accuracy: 0.9347
Epoch 6/100
53/53 [==============================] - 691s 13s/step - loss: 0.1185 - accuracy: 0.9532 - val_loss: 0.0875 - val_accuracy: 0.9720
Epoch 7/100
53/53 [==============================] - 27337s 525s/step - loss: 0.1002 - accuracy: 0.9605 - val_loss: 0.0809 - val_accuracy: 0.9

In [8]:
print(model.evaluate(valid_ds))

24/24 [==============================] - 24s 959ms/step - loss: 0.0244 - accuracy: 0.9893
[0.02444159984588623, 0.9893333315849304]


In [14]:
SAMPLES_TO_DISPLAY = 2

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))

for audios, labels in test_ds.take(1):
    # Obtener la señal FFT
    ffts = audio_to_fft(audios)
    # Predecir
    y_pred = model.predict(ffts)
    # Tomar muestras aleatorias
    rnd = np.random.randint(0, BATCH_SIZE, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        # Para cada muestra, imprimir la etiqueta verdadera y predicha, así como ejecutar la voz con su ruido
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        display(Audio(audios[index, :, :].squeeze(), rate=SAMPLING_RATE))

4/4 [==============================] - 4s 1s/step
Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau


Speaker: Jens_Stoltenberg	Predicted: Jens_Stoltenberg
